Questo notebook viene utilizzato per la lettura e analisi dei dati proveniente dal DB di WhoTeach

In [57]:
import pandas as pd
import math
import numpy as np

In [2]:
mdl_course = pd.read_csv("schemas/mdl_course.csv")
#,error_bad_lines=False)
mdl_course_categories = pd.read_csv("schemas/mdl_course_categories.csv")
mdl_course_sections = pd.read_csv("schemas/mdl_course_sections.csv")
full_text_section=  pd.read_csv("schemas/full_text_section.csv")
mdl_course_modules=  pd.read_csv("schemas/mdl_course_modules.csv")
mdl_url= pd.read_csv("schemas/mdl_url.csv")
mdl_metadata=pd.read_csv("schemas/mdl_metadata.csv")

In [3]:
#carico tutte le tabelle contenenti il nome delle risorse
#[1, 3, 4, 8, 9, 12, 13, 15, 16, 17, 20, 22]
#1
mdl_assign = pd.read_csv("schemas/mdl_assign.csv")
#3
mdl_book = pd.read_csv("schemas/mdl_book.csv")
#4
mdl_chat= pd.read_csv("schemas/mdl_chat.csv")
#8
mdl_folder = pd.read_csv("schemas/mdl_folder.csv")
#9
mdl_forum = pd.read_csv("schemas/mdl_forum.csv")
#12 
mdl_label = pd.read_csv("schemas/mdl_label.csv")
#13
mdl_lesson = pd.read_csv("schemas/mdl_lesson.csv")
#15
mdl_page = pd.read_csv("schemas/mdl_page.csv")
#16
mdl_quiz = pd.read_csv("schemas/mdl_quiz.csv")
#17
mdl_resource = pd.read_csv("schemas/mdl_resource.csv")
#20
mdl_url = pd.read_csv("schemas/mdl_url.csv")
#22
mdl_workshop= pd.read_csv("schemas/mdl_workshop.csv")

In [105]:
#contiene l'informazione dei corsi effettivi
#ha senso verificare il rate di ogni corso ? ci sono solo 15 valutazioni di 108 corsi
mdl_course.head()

,id,category,sortorder,fullname,shortname,idnumber,summary,summaryformat,format,showgrades,...,defaultgroupingid,lang,calendartype,theme,timecreated,timemodified,requested,enablecompletion,completionnotify,cacherev
0,1,0,1,WhoTeach Project,WhoTeach,NaN,"<p></p><p style=""margin: 5px 0px 0px; padding:...",0,site,1,...,0,NaN,NaN,NaN,1447767447,1507801566,0,0,0,1563799988
1,64,1,10011,Editoria Digitale 2017/2018,Editoria Digitale 2017/2018,NaN,"<p><span style=""color: rgb(85, 85, 85); font-f...",1,topics,1,...,0,NaN,NaN,NaN,1507115151,1510141827,0,0,0,1563799988
2,65,1,10010,Progetto multimediale 2016/2017,Progetto mm 2016/2017,NaN,NaN,1,weeks,1,...,0,NaN,NaN,NaN,1518347443,1518347443,0,0,0,1563799988
3,71,1,10009,"ITS Rizzoli - Programmatore Java, Android e Web","Programmatore Java, Android e Web",NaN,<p></p><h4>Profilo</h4><p>Tecnico per la proge...,1,weeks,1,...,0,it,NaN,NaN,1525962801,1526641246,0,0,0,1565625478
4,72,1,10008,INDUSTRY 4.0 E SMART FACTORY,IFTS 4.0,NaN,<p>Il corso&nbsp;è stato progettato per formar...,1,weeks,1,...,0,NaN,NaN,NaN,1525963746,1530885283,0,0,0,1563799988


In [10]:
set(mdl_metadata.query("Property =='keywords' and Id_course == 217 and Id_course_sections==1673")["Id_resource"])

{1742.0, 1743.0, 1744.0, 1745.0, 1746.0}

In [12]:
statistica = mdl_course_categories.query("id ==22")
corsi=mdl_course.query("category == 22")
corso_id_217=corsi.query("id == 217")
moduli=mdl_course_sections.query("course ==217")
modulo=moduli.query("id == 1673 and section == 2")
risorse=mdl_course_modules.query("section==1673")
# 1. get keywords for each res
for res_id in set(risorse['id']):
    print(res_id)

# 2. get name

risorsa= risorse.query("instance == 839")
#accesso alla risorsa effettiva
#instance in mdl_course_modules indica l'id della risorsa nella tabella riferita dal campo module
risorsa_effettiva=mdl_url.query("id == 839")
list(risorsa_effettiva.name)[0]
risorsa

1742
1743
1744
1745
1746


,id,course,module,instance,section,idnumber,added,score,indent,visible,...,visibleold,groupmode,groupingid,completion,completiongradeitemnumber,completionview,completionexpected,showdescription,availability,deletioninprogress
971,1744,217,20,839,1673,NaN,1422027596,0,0,1,...,1,0,0,0,NaN,0,0,0,NaN,0


In [13]:
# 1. get keywords for each res
a = []
for res_id in set(risorse['id']):
    print(res_id)
    a.append(mdl_metadata.loc[(mdl_metadata['Id_resource'] == res_id) & (mdl_metadata['Property'] == "keywords")]['Value'])
    break
    

1742


In [260]:
database=[]
categories={}
courses={}
sections={}
name_section={}

In [261]:
for index in range(len(mdl_course_categories)):
    categories[mdl_course_categories.id[index]]=mdl_course_categories.name[index]
categories

{1: 'Abilità informatiche',
 14: 'Competenze in Economia',
 15: 'Abilità Comunicative',
 16: 'Sviluppo Personale',
 20: 'Visione imprenditoriale',
 22: 'Probability and Statistics'}

In [262]:
#tutti i corsi all'interno della piattaforma
#id	category	sortorder	fullname	shortname
for index in range(len(mdl_course)):
        ctgy=mdl_course.category[index]
        if ctgy != 0:
            #print(categories[ctgy])
            '''
            if mdl_course.shortname[index]=="'nan'":
                courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index],'null']
            else:
                courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index],mdl_course.shortname[index]]
            '''
            courses[mdl_course.id[index]]=[categories[ctgy],mdl_course.fullname[index]]
len(courses)

107

In [263]:
#tutte le sezioni di tutti i corsi
#id	course	section	name
for index in range(len(mdl_course_sections)):
    crse=mdl_course_sections.course[index]
    if crse != 1:
        sections[mdl_course_sections.id[index]]=[courses[crse][0],courses[crse][1],mdl_course_sections.id[index]]

#len(full_text_section)
sections[1673]

['Probability and Statistics', 'Stochastic Estimation and Control', 1673]

In [264]:
risorse_totali={}
for key in sections.keys():
    risorse_totali[key]=mdl_course_modules.query("section == "+str(key)) 

temp=[]
#id module instance id_section section/information
for key in risorse_totali.keys():
    for index,row in risorse_totali[key].iterrows():
        my_list=[int(row.id),int(row.module),int(row.instance),int(row.section)]
        temp.append(my_list)

risorse_informazione=[]
for i in temp:
    t=i+sections[i[3]]
    risorse_informazione.append(t)

for i in risorse_informazione:
    module = i[1]
    line = i[2]
    if module==1:
        a=list(mdl_assign.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==3:
        a=list(mdl_book.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==4:
        a=list(mdl_chat.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==8:
        a=list(mdl_folder.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==9:
        a=list(mdl_forum.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==12:
        a=list(mdl_label.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==13:
        a=list(mdl_lesson.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==15:
        a=list(mdl_page.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==16:
        a=list(mdl_quiz.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==17:
        a=list(mdl_resource.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==20:
        a=list(mdl_url.query("id=="+str(line)).name)[0]
        i.append(a)
    elif module ==22:
        a=list(mdl_workshop.query("id=="+str(line)).name)[0]
        i.append(a)
    else: 
        print('error')



for i in risorse_informazione:
    del i[1:3]

keywords_metadata={}
for index,row in mdl_metadata.iterrows():
    if row.Property == 'keywords':
        if not(math.isnan(float(row.Id_resource))):
            keywords_metadata[int(row.Id_resource)]=[]
        
for index,row in mdl_metadata.iterrows():
    if row.Property == 'keywords':
        if not(math.isnan(float(row.Id_resource))):
            keywords_metadata[int(row.Id_resource)].append(row.Value)

risorse_informazione


[[483,
  998,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  998,
  'Forum News'],
 [484,
  999,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  999,
  'Docenti e Tutor\n    \n      \n        Docente: M....'],
 [485,
  999,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  999,
  'Forum del corso'],
 [486,
  1002,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  'Risorse online'],
 [487,
  1002,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  'Lezioni05/10/2017Presentazione del corso, modalità...'],
 [488,
  1002,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '10/10/2017Scenari della nuova editoria&nbsp;Lettur...'],
 [489,
  1002,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '12/10/2017Il libro digitale e le sue origini&nbsp;...'],
 [490,
  1002,
  'Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '17/10/2017Il mercato del libro&nbsp;Letture:&nb

In [265]:
#keywords_metadata è un dizionario key = id_risorsa e value = una lista delle keywords della risorsa
#risorse_informazione contiene tutte le informazioni per ogni risorsa 
for row in risorse_informazione:
    if row[0] in keywords_metadata.keys():
        temp=''
        for keyword in keywords_metadata[row[0]]:
            temp+=keyword+' ,'
        temp=temp[0:-2]
        row.append(temp)


In [266]:
#da 1690 a 1402 tenendo solo le risorse che hanno effettivamente delle keywords.
risorse_con_keywords=[]
for i in risorse_informazione:
    if len(i) > 5:
        del i[0]
        risorse_con_keywords.append(i)
print(risorse_con_keywords)

[[998, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 998, 'Forum News'], [999, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 999, 'Docenti e Tutor\n    \n      \n        Docente: M....'], [999, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 999, 'Forum del corso'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, 'Risorse online'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, 'Lezioni05/10/2017Presentazione del corso, modalità...'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, '10/10/2017Scenari della nuova editoria&nbsp;Lettur...'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, '12/10/2017Il libro digitale e le sue origini&nbsp;...'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, '17/10/2017Il mercato del libro&nbsp;Letture:&nbsp;...'], [1002, 'Abilità informatiche', 'Editoria Digitale 2017/2018', 1002, "19/10/2017Il mercato dell'editoria (lucidi prece

In [267]:
#tirare i nomi fuori da full_text_section poi inserirli nella posizione 2 in sections

for index in range(len(full_text_section)):
    name_section[full_text_section.id_section[index]]=full_text_section.name[index]

print(name_section[1132])
#full_text_section.query("id_section==1132")

''Review''


In [268]:
for row in risorse_con_keywords:
    if row[0] in name_section.keys():
        row.append(name_section[row[0]])
    #else:
        #row.append('null')
    del row[0]


In [269]:
#categoria - corso - id_sezione - risorsa - keywords
# le risorse fin'ora sono 1690 con valori mancanti
dataset_final=[]
for i in risorse_con_keywords:
    if len(i)==5:
        dataset_final.append(i)


[['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  'Risorse online',
  "''Random Variables and Distributions''"],
 ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  'Lezioni05/10/2017Presentazione del corso, modalità...',
  "''Random Variables and Distributions''"],
 ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '10/10/2017Scenari della nuova editoria&nbsp;Lettur...',
  "''Random Variables and Distributions''"],
 ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '12/10/2017Il libro digitale e le sue origini&nbsp;...',
  "''Random Variables and Distributions''"],
 ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  '17/10/2017Il mercato del libro&nbsp;Letture:&nbsp;...',
  "''Random Variables and Distributions''"],
 ['Abilità informatiche',
  'Editoria Digitale 2017/2018',
  1002,
  "19/10/2017Il mercato dell'editoria (lucidi precede...",
  "''Random Variables and Distributions''"],
 ['Abilità in

In [274]:
#numero di risorse effettive indipendentemente dal numero di corsi di appartenenza
num_risorse={}
for i in dataset_final:
    num_risorse[i[2]]=0
for i in dataset_final:
    num_risorse[i[2]]+=1
len(num_risorse)

396

In [277]:
categoria=[]
corso=[]
id_sezione=[]
risorsa=[]
keywords=[]
for i in dataset_final:
    categoria.append(i[0])
    corso.append(i[1])
    id_sezione.append(i[2])
    risorsa.append(i[3])
    keywords.append(i[4])


In [285]:
dataset = pd.DataFrame(columns=['id','categoria', 'corso', 'id_sezione','risorsa','keywords'])

In [297]:
dataset['id']=list(range(len(dataset_final)))
dataset['categoria']=categoria
dataset['corso']=corso
dataset['id_sezione']=id_sezione
dataset['risorsa']=risorsa
dataset['keywords']=keywords
dataset.head()

,id,categoria,corso,id_sezione,risorsa,keywords
0,0,Abilità informatiche,Editoria Digitale 2017/2018,1002,Risorse online,''Random Variables and Distributions''
1,1,Abilità informatiche,Editoria Digitale 2017/2018,1002,"Lezioni05/10/2017Presentazione del corso, moda...",''Random Variables and Distributions''
2,2,Abilità informatiche,Editoria Digitale 2017/2018,1002,10/10/2017Scenari della nuova editoria&nbsp;Le...,''Random Variables and Distributions''
3,3,Abilità informatiche,Editoria Digitale 2017/2018,1002,12/10/2017Il libro digitale e le sue origini&n...,''Random Variables and Distributions''
4,4,Abilità informatiche,Editoria Digitale 2017/2018,1002,17/10/2017Il mercato del libro&nbsp;Letture:&n...,''Random Variables and Distributions''


In [299]:
dataset.to_csv('data_wt.csv', index=False)